In [33]:
import pandas as pd
import os
import geopandas as gpd
import shapely
from shapely.ops import nearest_points
from tqdm import tqdm, trange
import kml2geojson
import json
from shapely.geometry import Point

In [34]:
files=os.listdir(r'C:\Users\poletaev_im\Downloads\Маршруты_для_Москвы')
files=pd.DataFrame(files, columns=['filename'])
files

,filename
0,волкоский театр_20190320223004


In [10]:
stops = gpd.read_file('Остановки.shp', encoding='utf-8')
unary_union = stops.unary_union
stops

,STOP_ID,STOP_NAME,X,Y,geometry
0,458,По требованию,36.100510,52.951440,POINT (36.10051 52.95144)
1,459,Кромская улица,36.018430,52.927712,POINT (36.01843 52.927712)
2,460,Кромская улица,36.061279,52.975694,POINT (36.061279 52.975694)
3,1,Медведевский лес,36.171463,53.022736,POINT (36.171463 53.022736)
4,2,Посёлок Булгаковский,36.173137,53.022784,POINT (36.173137 53.022784)
5,3,Торгово-развлекательный центр РИО,36.160945,53.015622,POINT (36.160945 53.015622)
6,4,Зона отдыха,36.178736,52.949687,POINT (36.178736 52.949687)
7,5,Зона отдыха,36.179115,52.949525,POINT (36.179115 52.949525)
8,6,362-й километр,36.179437,52.965982,POINT (36.179437 52.965982)
9,7,362-й километр,36.178780,52.965467,POINT (36.17878 52.965467)


In [11]:
def nearest(row, geom_union, df1, df2, geom1_col='geometry', geom2_col='geometry', src_column=None):
    """Find the nearest point and return the corresponding value from specified column."""
    # Find the geometry that is closest
    nearest = df2[geom2_col] == nearest_points(row[geom1_col], geom_union)[1]
    # Get the corresponding value from df2 (matching is based on the geometry)
    value = df2[nearest][src_column].get_values()[0]
    return value

In [12]:
from tqdm import tqdm_notebook as tqdm
all_kml=gpd.GeoDataFrame()
for y in tqdm(range(len(files))):
    file=r'C:\Users\poletaev_im\Downloads\Маршруты_для_Москвы\%s' % (files.filename[y])
    
    kml2geojson.main.convert(file,'C:\\Users\\poletaev_im\\nearest_stop_id_from_kml\\' , separate_folders=True)
    stops_kml = gpd.read_file('Маркеры.geojson', layer = 0)
    with open('Маршруты.geojson') as f:
        data = json.load(f,  parse_int=True)
    coo =  pd.DataFrame(data['features'][1]['geometry']['coordinates'], columns = ['lon', 'lat','no'])
    times = pd.DataFrame(data['features'][1]['properties']['times'], columns = ['times'])
    geometry = [Point(xy) for xy in zip(coo.lon, coo.lat)]
    coo = coo.drop(['lon', 'lat'], axis=1)
    coord = gpd.GeoDataFrame(coo, geometry=geometry)
    coord = coord.join(times)
    kml_file = gpd.sjoin(stops_kml, coord, how = 'left', op = 'intersects')
    kml_file = kml_file.drop(['styleUrl', 'color','index_right'], axis=1)
    
    kml_file['nearest_id'] = kml_file.apply(nearest, geom_union=unary_union, df1=kml_file, df2=stops, geom1_col='geometry', src_column='STOP_ID', axis=1)
    
    kml_file['route_date_name']=files.filename[y]
    kml_file = kml_file.reset_index()
    all_kml=all_kml.append(kml_file)
    
all_kml = all_kml.reset_index(drop=True)
all_kml

A Jupyter Widget

C:\Users\poletaev_im\AppData\Local\Continuum\anaconda3\lib\site-packages\geopandas\tools\sjoin.py:44: UserWarning: CRS of frames being joined does not match!
  warn('CRS of frames being joined does not match!')


,index,name,geometry,no,times,nearest_id,route_date_name
0,0,Веселая Слобода,POINT (36.03519439697266 52.96808624267578),173.0,2018-12-01T09:31:26.000Z,451,01_В696ЕА 01.12.2018 12_31_26 - 01.12.2018 14_...
1,1,Стадион им. Ленина,POINT (36.04659271240234 52.97294235229492),180.0,2018-12-01T09:34:40.000Z,431,01_В696ЕА 01.12.2018 12_31_26 - 01.12.2018 14_...
2,2,ТЦ Тургеневский,POINT (36.05345153808594 52.97116851806641),171.0,2018-12-01T09:36:24.000Z,429,01_В696ЕА 01.12.2018 12_31_26 - 01.12.2018 14_...
3,3,Атолл,POINT (36.05547332763672 52.97021865844727),185.0,2018-12-01T09:38:30.000Z,428,01_В696ЕА 01.12.2018 12_31_26 - 01.12.2018 14_...
4,4,1-я Посадская ул-ца,POINT (36.0646858215332 52.9599494934082),154.0,2018-12-01T09:43:17.000Z,424,01_В696ЕА 01.12.2018 12_31_26 - 01.12.2018 14_...
5,5,1-я Посадская улица,POINT (36.0666389465332 52.95998001098633),159.0,2018-12-01T09:44:49.000Z,132,01_В696ЕА 01.12.2018 12_31_26 - 01.12.2018 14_...
6,6,Школа 26,POINT (36.07155990600586 52.95926666259766),156.0,2018-12-01T09:49:00.000Z,134,01_В696ЕА 01.12.2018 12_31_26 - 01.12.2018 14_...
7,7,Супермаркет Апельсин,POINT (36.07390594482422 52.95436096191406),157.0,2018-12-01T09:50:44.000Z,137,01_В696ЕА 01.12.2018 12_31_26 - 01.12.2018 14_...
8,8,Песковская,POINT (36.07487487792969 52.9520149230957),154.0,2018-12-01T09:51:13.000Z,184,01_В696ЕА 01.12.2018 12_31_26 - 01.12.2018 14_...
9,9,Городская улица,POINT (36.07683944702148 52.95042037963867),152.0,2018-12-01T09:51:31.000Z,186,01_В696ЕА 01.12.2018 12_31_26 - 01.12.2018 14_...


In [32]:
import time
from datetime import datetime

In [31]:
all_kml['date'], all_kml['time'] = all_kml.times.str.split('T',1).str
all_kml['time'] = all_kml.time.str.split('Z',1,expand=True)
all_kml = all_kml.dropna(0)
all_kml

,index,name,geometry,no,times,nearest_id,route_date_name,data,time,date
0,0,Веселая Слобода,POINT (36.03519439697266 52.96808624267578),173.0,2018-12-01T09:31:26.000Z,451,01_В696ЕА 01.12.2018 12_31_26 - 01.12.2018 14_...,2018-12-01,09:31:26.000,2018-12-01
1,1,Стадион им. Ленина,POINT (36.04659271240234 52.97294235229492),180.0,2018-12-01T09:34:40.000Z,431,01_В696ЕА 01.12.2018 12_31_26 - 01.12.2018 14_...,2018-12-01,09:34:40.000,2018-12-01
2,2,ТЦ Тургеневский,POINT (36.05345153808594 52.97116851806641),171.0,2018-12-01T09:36:24.000Z,429,01_В696ЕА 01.12.2018 12_31_26 - 01.12.2018 14_...,2018-12-01,09:36:24.000,2018-12-01
3,3,Атолл,POINT (36.05547332763672 52.97021865844727),185.0,2018-12-01T09:38:30.000Z,428,01_В696ЕА 01.12.2018 12_31_26 - 01.12.2018 14_...,2018-12-01,09:38:30.000,2018-12-01
4,4,1-я Посадская ул-ца,POINT (36.0646858215332 52.9599494934082),154.0,2018-12-01T09:43:17.000Z,424,01_В696ЕА 01.12.2018 12_31_26 - 01.12.2018 14_...,2018-12-01,09:43:17.000,2018-12-01
5,5,1-я Посадская улица,POINT (36.0666389465332 52.95998001098633),159.0,2018-12-01T09:44:49.000Z,132,01_В696ЕА 01.12.2018 12_31_26 - 01.12.2018 14_...,2018-12-01,09:44:49.000,2018-12-01
6,6,Школа 26,POINT (36.07155990600586 52.95926666259766),156.0,2018-12-01T09:49:00.000Z,134,01_В696ЕА 01.12.2018 12_31_26 - 01.12.2018 14_...,2018-12-01,09:49:00.000,2018-12-01
7,7,Супермаркет Апельсин,POINT (36.07390594482422 52.95436096191406),157.0,2018-12-01T09:50:44.000Z,137,01_В696ЕА 01.12.2018 12_31_26 - 01.12.2018 14_...,2018-12-01,09:50:44.000,2018-12-01
8,8,Песковская,POINT (36.07487487792969 52.9520149230957),154.0,2018-12-01T09:51:13.000Z,184,01_В696ЕА 01.12.2018 12_31_26 - 01.12.2018 14_...,2018-12-01,09:51:13.000,2018-12-01
9,9,Городская улица,POINT (36.07683944702148 52.95042037963867),152.0,2018-12-01T09:51:31.000Z,186,01_В696ЕА 01.12.2018 12_31_26 - 01.12.2018 14_...,2018-12-01,09:51:31.000,2018-12-01


In [13]:
all_kml.to_csv('all_kml.csv', sep=';', encoding='cp1251', index=False)